<a href="https://colab.research.google.com/github/MykolaPrykhodko/MykolaPrykhodko/blob/main/%D0%94%D0%B5%D0%BD%D0%B8%D1%81_Buget_FOP_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Програма розрахунку бюджетних показників ФОП АТ "Украгазбанка"**

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

## Підключення хмарного середовища зберігання довідників для розрахунку

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Для читання потрібна певна авторізація

###Імпортування бібліотек для аутентифікації та встановлення зв'язку із Таблицями.

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

### Відкриття файла із переліком персонала для обчислювання за посіланням (url)

In [4]:
path_in ='/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/staff.xlsx'

### Отримані дані перетворюємо на датафрейм, як дінамічну структуру даних для подальшого доповнення результатами розрахунків (як приклад - в консоль роздруковано перших 20 значень)

In [5]:
# df_staff = pd.read_excel(path_in).astype({'tabNum': str}, errors='raise')
df_staff = pd.read_excel(path_in, dtype={'tabNum': str, 'Name': str, 'CFOcode': str, 'CFOname': str})

In [6]:
df_staff.loc[df_staff['tabNum'] == '!13853']

,tabNum,Name,CFOcode,CFOname
3596,!13853,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків


In [7]:
df_staff.loc[df_staff['tabNum'] == '!13853']

,tabNum,Name,CFOcode,CFOname
3596,!13853,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків


In [8]:
df_staff_for_merge = pd.read_excel(path_in).astype({"tabNum": str}, errors='raise')

In [9]:
df_staff.loc[df_staff['tabNum'] == '!13853']

,tabNum,Name,CFOcode,CFOname
3596,!13853,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків


In [10]:
df_staff.loc[df_staff['tabNum'] == '!13853']

,tabNum,Name,CFOcode,CFOname
3596,!13853,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків


## Допоміжні функції розрахунків

In [11]:
def avg_salary(path_in):
  p_3 = pd.read_excel(path_in)

  def calculate(num_tab):
    x = 1
    y = 7
    lst = []
    while y <= len(p_3.columns):
        avg = sum(p_3.iloc[num_tab, x:y])/180
        lst.append(avg)
        x += 1
        y += 1
    return lst
  
  global_lst = []
  for i in range(len(p_3)):
    global_lst.append(calculate(i))
    
  df = pd.DataFrame(global_lst)

  finish = pd.concat([p_3['tabNum'], df], axis=1).iloc[:,0:13]
  finish.rename(columns={0: 'січень'
  , 1: 'лютий'
  , 2: 'березень'
  , 3: 'квітень'
  , 4: 'травень'
  , 5: 'червень'
  , 6: 'липень'
  , 7: 'серпень'
  , 8: 'вересень'
  , 9: 'жовтень'
  , 10: 'листопад'
  , 11: 'грудень'}, inplace=True)
  finish=pd.melt(finish, id_vars='tabNum', value_vars=['січень',
                                                        'лютий',
                                                        'березень',
                                                        'квітень',
                                                        'травень',
                                                        'червень',
                                                        'липень',
                                                        'серпень',
                                                        'вересень',
                                                        'жовтень',
                                                        'листопад',
                                                        'грудень'])
  
  finish = finish.rename(columns={'variable': 'period', 'value': 'avg_salary'})
  finish = finish.astype({"tabNum": str}, errors='raise') 
  return finish

In [12]:
def f_54110(path_in):
    # Считывание файла справочника
    df_salary = pd.read_excel(path_in
                              , sheet_name='_54110'
                              , usecols='A,C:N'
                              , converters={'Таб №': str,
                                            'січень': float,
                                            'лютий': float,
                                            'березень': float,
                                            'квітень': float,
                                            'травень': float,
                                            'червень': float,
                                            'липень': float,
                                            'серпень': float,
                                            'вересень': float,
                                            'жовтень': float,
                                            'листопад': float,
                                            'грудень': float})
    # Преобразование таблицы из широкой в длинную для связывания с итоговой
    df_salary_unpivot = pd.melt(df_salary, id_vars='Таб. №', value_vars=['січень',
                                                                         'лютий',
                                                                         'березень',
                                                                         'квітень',
                                                                         'травень',
                                                                         'червень',
                                                                         'липень',
                                                                         'серпень',
                                                                         'вересень',
                                                                         'жовтень',
                                                                         'листопад',
                                                                         'грудень'])
    df_salary_unpivot = df_salary_unpivot.astype({"Таб. №": str}, errors='raise')
    return df_salary_unpivot

In [13]:
def f_54118(path_in):
    # Считывание файла справочника
    df_vacantion = pd.read_excel(path_in,
                                 sheet_name='_54118',
                                 usecols='A,C:N',
                                 converters={'Таб №': str,
                                             'січень': float,
                                             'лютий': float,
                                             'березень': float,
                                             'квітень': float,
                                             'травень': float,
                                             'червень': float,
                                             'липень': float,
                                             'серпень': float,
                                             'вересень': float,
                                             'жовтень': float,
                                             'листопад': float,
                                             'грудень': float})

    # Преобразование таблицы из широкой в длинную для связывания с итоговой
    df_vacantion_unpivot = pd.melt(df_vacantion, id_vars='Таб. №', value_vars=['січень',
                                                                               'лютий',
                                                                               'березень',
                                                                               'квітень',
                                                                               'травень',
                                                                               'червень',
                                                                               'липень',
                                                                               'серпень',
                                                                               'вересень',
                                                                               'жовтень',
                                                                               'листопад',
                                                                               'грудень'])

    df_salary_unpivot = df_vacantion_unpivot.astype({"Таб. №": str}, errors='raise')
    df_salary_unpivot = df_salary_unpivot['value'].fillna(0)
    
    return df_vacantion_unpivot

In [14]:
# Додаткова заробітна плата
def f_54112(path_in):
    # Считывание файла справочника
    df_add_salary = pd.read_excel(path_in
                                  , sheet_name='_54112'
                                  , usecols='B,G'
                                  , converters={'Таб. №': str,
                                                'итог': float})
    return df_add_salary

In [15]:
# Додаткова заробітна плата 5.4.1.1.3.3. премія за підсумками роботи Банку (за фінансовий рік)
def f_541133(path_in):
    # Считывание файла справочника
    df_add_salary_1 = pd.read_excel(path_in
                                    , sheet_name='_541133'
                                    , usecols='A,C:N').fillna(0)
    # Переворот таблицы из широкой в длинную
    df_add_salary_1 = pd.melt(df_add_salary_1, id_vars='Таб. №', value_vars=['січень',
                                                                             'лютий',
                                                                             'березень',
                                                                             'квітень',
                                                                             'травень',
                                                                             'червень',
                                                                             'липень',
                                                                             'серпень',
                                                                             'вересень',
                                                                             'жовтень',
                                                                             'листопад',
                                                                             'грудень'])
    df_add_salary_1 = df_add_salary_1.astype({"Таб. №": str}, errors='raise')
    
    df_add_salary_1 = df_add_salary_1.rename(columns={'Таб. №': 'tabNum',
                                                      'variable': 'period',
                                                      'value': 'add_salary_1'})

    
    return df_add_salary_1

In [16]:
# Додаткова заробітна плата 5.4.1.1.3.5. інші премії
def f_541135(path_in):
    # Считывание файла справочника
    df_add_salary_2 = pd.read_excel(path_in
                                    , sheet_name='_541135'
                                    , usecols='A,C:N').fillna(0)
    # Переворот таблицы из широкой в длинную
    df_add_salary_2 = pd.melt(df_add_salary_2, id_vars='Таб. №', value_vars=['січень',
                                                                             'лютий',
                                                                             'березень',
                                                                             'квітень',
                                                                             'травень',
                                                                             'червень',
                                                                             'липень',
                                                                             'серпень',
                                                                             'вересень',
                                                                             'жовтень',
                                                                             'листопад',
                                                                             'грудень'])
    df_add_salary_2 = df_add_salary_2.astype({"Таб. №": str}, errors='raise')
    df_add_salary_2 = df_add_salary_2.rename(columns={'Таб. №': 'tabNum',
                                                      'variable': 'period',
                                                      'value': 'add_salary_2'})

    return df_add_salary_2

In [17]:
# 5.4.1.1.4. трудові угоди
def f_54114(path_in):
    # Считывание файла справочника
    df_work_argmnt = pd.read_excel(path_in, sheet_name='_54114')

    df_work_argmnt = pd.melt(df_work_argmnt, id_vars='Таб. №', value_vars=['січень',
                                                                           'лютий',
                                                                           'березень',
                                                                           'квітень',
                                                                           'травень',
                                                                           'червень',
                                                                           'липень',
                                                                           'серпень',
                                                                           'вересень',
                                                                           'жовтень',
                                                                           'листопад',
                                                                           'грудень'])
    
    df_work_argmnt = df_work_argmnt.astype({"Таб. №": str}, errors='raise')

    df_work_argmnt = df_work_argmnt.rename(columns={'Таб. №': 'tabNum',
                                                    'variable': 'period',
                                                    'value': 'bug_work_argmnt'}).fillna(0)
    return df_work_argmnt

In [18]:
# 5.4.1.1.5. резерв відпусток 7кл.
def f_54115_soc(path_in):
    # Считывание файла справочника
    df_work_argmnt = pd.read_excel(path_in, sheet_name='_54115_soc', usecols='A,C:N').fillna(0)
    
    df_work_argmnt = pd.melt(df_work_argmnt, id_vars='Таб. №', value_vars=['січень',
                                                                           'лютий',
                                                                           'березень',
                                                                           'квітень',
                                                                           'травень',
                                                                           'червень',
                                                                           'липень',
                                                                           'серпень',
                                                                           'вересень',
                                                                           'жовтень',
                                                                           'листопад',
                                                                           'грудень'])
    
    df_work_argmnt = df_work_argmnt.astype({"Таб. №": str}, errors='raise')
    
    df_work_argmnt = df_work_argmnt.rename(columns={'Таб. №': 'tabNum',
                                                    'variable': 'period',
                                                    'value': 'bug_work_argmnt_soc'}).fillna(0)
    
    return df_work_argmnt

In [19]:
def f_5413(path_in):
    # Считывание файла справочника
    df_med_help = pd.read_excel(path_in, sheet_name='_5413', usecols='A:M', converters={'Місяць': str,
                                                                                        'січень': float,
                                                                                        'лютий': float,
                                                                                        'березень': float,
                                                                                        'квітень': float,
                                                                                        'травень': float,
                                                                                        'червень': float,
                                                                                        'липень': float,
                                                                                        'серпень': float,
                                                                                        'вересень': float,
                                                                                        'жовтень': float,
                                                                                        'листопад': float,
                                                                                        'грудень': float})

    df_med_help = pd.melt(df_med_help, id_vars='Місяць', value_vars=['січень',
                                                                     'лютий',
                                                                     'березень',
                                                                     'квітень',
                                                                     'травень',
                                                                     'червень',
                                                                     'липень',
                                                                     'серпень',
                                                                     'вересень',
                                                                     'жовтень',
                                                                     'листопад',
                                                                     'грудень'])
    df_med_help = df_med_help.rename(columns={'variable': 'period', 'value': 'bug_med_help'})
    del df_med_help['Місяць']

    return df_med_help

In [20]:
# 5.4.1.1.7. індексація
def f_54117(path_in):
    # Считывание файла справочника
    df_index = pd.read_excel(path_in, sheet_name='_54117', usecols='A:M')

    df_index = pd.melt(df_index, id_vars='Місяць', value_vars=['січень',
                                                               'лютий',
                                                               'березень',
                                                               'квітень',
                                                               'травень',
                                                               'червень',
                                                               'липень',
                                                               'серпень',
                                                               'вересень',
                                                               'жовтень',
                                                               'листопад',
                                                               'грудень'])
    df_index = df_index.rename(columns={'variable': 'period', 'value': 'bug_index'})
    del df_index['Місяць']

    return df_index

## **Розрахунок:**

### Щоб отримати щомісячний диапазон планування необхідно додати місяний розподіл:

In [21]:
months = ['січень', 'лютий', 'березень', 'квітень', 'травень', 'червень', 'липень', 'серпень', 'вересень', 'жовтень', 'листопад', 'грудень']

In [22]:
df_staff_month = []
for index, row in df_staff.iterrows():
  for l in months:
    df_staff_month.append({
        'tabNum': str(row['tabNum']),
        'period': str(l)})
df_staff = pd.DataFrame(df_staff_month).reindex()

In [23]:
df_staff.loc[df_staff['tabNum'] == '!13853']

,tabNum,period
43152,!13853,січень
43153,!13853,лютий
43154,!13853,березень
43155,!13853,квітень
43156,!13853,травень
43157,!13853,червень
43158,!13853,липень
43159,!13853,серпень
43160,!13853,вересень
43161,!13853,жовтень


In [24]:
df_staff = pd.merge(left=df_staff, right=df_staff_for_merge, left_on='tabNum', right_on='tabNum', how='left')

In [25]:
df_staff.loc[df_staff['tabNum'] == '!13853']

,tabNum,period,Name,CFOcode,CFOname
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків


In [26]:
#  заміна порожніх значень NaN на пробіл
df_staff['Name'] = df_staff['Name'].fillna('')

In [27]:
df_staff.loc[df_staff['tabNum'] == '!13853']

,tabNum,period,Name,CFOcode,CFOname
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків


### Доповнення розрахунком середньої зп (період розрахунку попрердні перед розрахунковим 6 місяців) відповідно до наданого файлу [(staff_in_avg.xlsx)](https://docs.google.com/spreadsheets/d/1gVaC_SLXgcQyjoJW7RNPTRR8Zn2Aec2Z/edit#gid=279750554)

In [28]:
path_in ='/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/staff_in_avg.xlsx'

до кожного табельного номеру розраховується середня зп за останні 6 місяців звітнього року за допомогою функції ***avg_salary***:

In [29]:
df_test = avg_salary(path_in)

In [30]:
df_test.head()

,tabNum,period,avg_salary
0,!8,січень,33.333333
1,!11,січень,33.333333
2,!29,січень,33.333333
3,!31,січень,33.333333
4,!42,січень,33.333333


In [31]:
df_staff = pd.merge(df_staff, avg_salary(path_in), how='left', left_on=['tabNum', 'period'], right_on=['tabNum', 'period'])

In [32]:
df_staff.loc[df_staff['tabNum'] == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333


In [33]:
#  заміна пустих значень NaN на 0 по колонці avg_salary
df_staff['avg_salary'] = df_staff['avg_salary'].fillna(0)

In [34]:
df_staff.loc[df_staff['tabNum'] == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333


### Додавання планових зп на бюжетний рік, що надані в файлі [data_54110.xlsx](https://docs.google.com/spreadsheets/d/1wmMN12-fZAtHARUrTNq21fODYgDyiwSB/edit?usp=drive_web&ouid=114658159574667661980&rtpof=true) за статею **5.4.1.1.0. "Основна заробітна плата (оклади) - бюджет"**

In [35]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54110.xlsx'

In [36]:
# проміжна таблиця для приведення у відповідність назви столбців
df_staff_for_merge = f_54110(path_in).rename(columns={'Таб. №': 'tabNum', 'variable': 'period', 'value': 'bug_salary'})
# доповнення колонкою 'bug_salary' що відповідає статті 
df_staff = pd.merge(df_staff, df_staff_for_merge, how='left', left_on=['tabNum', 'period'], right_on=['tabNum', 'period'])
# заповнення пустых значень NaN на 0
df_staff['bug_salary'] = df_staff['bug_salary'].fillna(0)

In [37]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0


### Додавання залишків відпусток (файл [data_54118.xlsx](https://docs.google.com/spreadsheets/d/1Y9uddy19MXbpjJitglE_0zqERrL2ism5/edit?usp=drive_web&ouid=114658159574667661980&rtpof=true)), як додаткову колонку ('vacantion') для використання в подальшому розрахунку

In [38]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54118.xlsx'

In [39]:
df_staff_for_merge = f_54118(path_in).rename(columns={'Таб. №': 'tabNum', 'variable': 'period', 'value': 'vacantion'})
df_staff_for_merge['vacantion'] = df_staff_for_merge['vacantion'].fillna(0)
df_staff_for_merge = df_staff_for_merge.astype({"tabNum": str}, errors='raise')

In [40]:
df_staff_for_merge.head()

,tabNum,period,vacantion
0,!8587,січень,10.0
1,!10786,січень,10.0
2,!10581,січень,10.0
3,!15087,січень,0.0
4,!12774,січень,0.0


In [41]:
df_staff = pd.merge(df_staff, df_staff_for_merge, how='left', left_on=['tabNum', 'period'], right_on=['tabNum','period']).fillna(0)

In [42]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0


### Показник **5.4.1.1.8. "Виплачені відпустки (3-й клас) - бюджет"** визначається, як добуток:
\begin{gather*}
{Виплачені\ відпустки (місяць)}_{бюджет} = ЗП\ середня(місяць)_{грн} \cdot Кількість(місяць)_{дн}
\end{gather*}

In [43]:
df_staff['bug_vacantion'] = df_staff['avg_salary'] * df_staff['vacantion']

In [44]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000


### Показник **5.4.1.1.1. "Основна заробітна плата"** розраховується, як різниця:
\begin{gather*}
{Основна\ заробітна\ плата(місяць)}_{бюджет} = {Основна\ заробітна\ плата/оклади\ (місяць)}_{бюджет} - {Виплачені\ відпустки (місяць)}_{бюджет} 
\end{gather*}

In [45]:
df_staff['bug_gen_salary'] = df_staff['bug_salary'] - df_staff['bug_vacantion']

In [46]:
df_staff['bug_vacantion']

0          0.000000
1          0.000000
2        233.333333
3          0.000000
4          0.000000
            ...    
44767      0.000000
44768      0.000000
44769      0.000000
44770      0.000000
44771      0.000000
Name: bug_vacantion, Length: 44772, dtype: float64

In [47]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000


### Показник **5.4.1.1.2. Додаткова заробітна плата**, визначається з файлу [data_54112.xlsx](https://docs.google.com/spreadsheets/d/13VZVPmeKFVC6Mh6TAB0EP9gQjuFTDc6m/edit#gid=2127744074)

In [48]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54112.xlsx'

In [49]:
#
df_staff_for_merge = f_54112(path_in).rename(columns={'Таб. №': 'tabNum', 'итог': 'bug_add_salary'})
df_staff_for_merge['bug_add_salary'] = df_staff_for_merge['bug_add_salary'].fillna(0)
df_staff = df_staff.merge(df_staff_for_merge, on='tabNum', how='left').fillna(0)

In [50]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333,1550.0
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0


### Показник **5.4.1.1.3.3. премія за підсумками роботи Банку (за фінансовий рік)** визначається з файлу [data_541133.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [51]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_541133.xlsx'

In [52]:
df_staff = pd.merge(df_staff, f_541133(path_in), how='left', left_on=['tabNum', 'period'], right_on=['tabNum', 'period']).fillna(0)

In [53]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333,1550.0,0.0
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0


###Показник **5.4.1.1.3.5. Інші премії** визначається з файлу [data_541135.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [54]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_541135.xlsx'

In [55]:
df_staff = pd.merge(df_staff, f_541135(path_in), how='left', left_on=['tabNum', 'period'],
                    right_on=['tabNum',
                              'period']).fillna(0)

In [56]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333,1550.0,0.0,55555.0
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0


###Показник **5.4.1.1.5. Резерв відпусток (7 кл.)** визначається за допомогою довідника [data_54115.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [57]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54115.xlsx'

In [58]:
# Завантаження довідника
df_add_vacant = pd.read_excel(path_in, sheet_name='_54115', usecols='A,E').fillna(0)
# Приведення у відповідність назв вимірів
df_add_vacant = df_add_vacant.rename(columns={'Таб. №': 'tabNum', 'Днів': 'bug_vac_reserv_1'})
df_add_vacant['tabNum'] = df_add_vacant['tabNum'].astype('str').replace('.0', '')
# Додавання виміру до кінцевого датасета
df_staff = df_staff.merge(df_add_vacant, on='tabNum', how='left').fillna(0)

In [59]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2,bug_vac_reserv_1
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333,1550.0,0.0,55555.0,2.0
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0


In [60]:
# Завантаження довідника соціальних відпусток
df_add_vacant_soc = f_54115_soc(path_in)
df_staff = pd.merge(df_staff, f_54115_soc(path_in), how='left', left_on=['tabNum', 'period'], right_on=['tabNum',
                                                                                                     'period']).fillna(0)

In [61]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2,bug_vac_reserv_1,bug_work_argmnt_soc
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,10.0
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333,1550.0,0.0,55555.0,2.0,0.0
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,6.0
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,3.0
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0


In [62]:
df_staff['bug_vac_reserv'] = df_staff['avg_salary'] * (df_staff['bug_vac_reserv_1'] + df_staff['bug_work_argmnt_soc'])

In [63]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2,bug_vac_reserv_1,bug_work_argmnt_soc,bug_vac_reserv
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,10.0,400.000000
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333,1550.0,0.0,55555.0,2.0,0.0,66.666667
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,6.0,266.666667
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,3.0,166.666667
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667


### Показник **5.4.1.1.4. трудові угоди** визначається за допомогою довідника [data_54114.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [64]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54114.xlsx'

In [65]:
df_staff = pd.merge(df_staff, f_54114(path_in), how='left', left_on=['tabNum', 'period'], right_on=['tabNum', 'period']).fillna(0)

In [66]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2,bug_vac_reserv_1,bug_work_argmnt_soc,bug_vac_reserv,bug_work_argmnt
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,10.0,400.000000,0.0
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333,1550.0,0.0,55555.0,2.0,0.0,66.666667,0.0
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,6.0,266.666667,0.0
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,3.0,166.666667,0.0
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0


### Показник **5.4.1.3. мат. допомога і інші утримання** визначається за допомогою довідника [data_5413.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [67]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_5413.xlsx'

In [68]:
# Додавання виміру мат.допомоги до кінцевого датасету
df_staff = df_staff.merge(f_5413(path_in), on='period', how='left').fillna(0)

In [69]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2,bug_vac_reserv_1,bug_work_argmnt_soc,bug_vac_reserv,bug_work_argmnt,bug_med_help
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,565000.0
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,10.0,400.000000,0.0,565000.0
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333,1550.0,0.0,55555.0,2.0,0.0,66.666667,0.0,3627500.0
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,6.0,266.666667,0.0,565000.0
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,565000.0
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,627500.0
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,3.0,166.666667,0.0,560000.0
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,560000.0
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,622500.0
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,560000.0


### Проміжний розрахунок індексації за допомогою файлу [data_54117.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [70]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54117.xlsx'

In [71]:
# Додавання значення індексу в розрахункову таблицю
df_staff = df_staff.merge(f_54117(path_in), on='period', how='left').fillna(0)

### Показник **5.4.1.1.7. Індексація** (остаточний розрахунок):

In [72]:
# ???? уточнить формулу с учетом 5.4.1.3.	мат. допомога і інші утримання
df_staff['bug_indexing'] = (df_staff['bug_salary']
                            + df_staff['bug_add_salary']
                            + df_staff['add_salary_1']
                            + df_staff['add_salary_2']
                            + df_staff['bug_work_argmnt']
                            + df_staff['bug_med_help']) * df_staff['bug_index']

In [73]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2,bug_vac_reserv_1,bug_work_argmnt_soc,bug_vac_reserv,bug_work_argmnt,bug_med_help,bug_index,bug_indexing
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,565000.0,0.00832,5256.82560
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,10.0,400.000000,0.0,565000.0,0.00820,5181.00600
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333,1550.0,0.0,55555.0,2.0,0.0,66.666667,0.0,3627500.0,0.00805,30186.57425
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,6.0,266.666667,0.0,565000.0,0.00790,4991.45700
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,565000.0,0.00780,4928.27400
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,627500.0,0.00770,5346.34100
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,1550.0,0.0,0.0,2.0,3.0,166.666667,0.0,560000.0,0.00770,4826.59100
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,560000.0,0.00780,4889.27400
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,622500.0,0.00790,5445.70700
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,1550.0,0.0,0.0,2.0,0.0,66.666667,0.0,560000.0,0.00790,4951.95700


### Показник **5.4.1.2.1. Обов'язкове державне пенсійне та соціальне страхування** 

In [74]:
var_54117 = 6500*15     # ХЗ что за переменная
df_staff['bug_gov_insurance'] = np.where((df_staff['bug_salary']
                                          + df_staff['bug_add_salary']
                                          + df_staff['add_salary_1']
                                          + df_staff['add_salary_2']
                                          + df_staff['bug_work_argmnt']
                                          + df_staff['bug_med_help']) <= var_54117,
                                         (df_staff['bug_salary']
                                          + df_staff['bug_add_salary']
                                          + df_staff['add_salary_1']
                                          + df_staff['add_salary_2']
                                          + df_staff['bug_work_argmnt']
                                          + df_staff['bug_med_help']) * 0.22,
                                         var_54117 * 0.22) + df_staff['bug_vac_reserv'] * 0.22

### Показник **5.4.1.5.8. профспілкові відрахування**

In [75]:
# Визначення видношення до профспілки
df_staff['bug_labour_union'] = np.where(df_staff['tabNum'].str.startswith(('труд'), na=False), 0, (df_staff['bug_salary']
                                                                                                   + df_staff['bug_add_salary']
                                                                                                   + df_staff['add_salary_1']
                                                                                                   + df_staff['add_salary_2']
                                                                                                   + df_staff['bug_vac_reserv']
                                                                                                   + df_staff['bug_indexing']
                                                                                                   + df_staff['bug_med_help']
                                                                                                   + df_staff['bug_gov_insurance']) * 0.05)

In [76]:
df_staff.loc[df_staff.tabNum == '!13853']

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,...,add_salary_2,bug_vac_reserv_1,bug_work_argmnt_soc,bug_vac_reserv,bug_work_argmnt,bug_med_help,bug_index,bug_indexing,bug_gov_insurance,bug_labour_union
43152,!13853,січень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,...,0.0,2.0,0.0,66.666667,0.0,565000.0,0.00832,5256.82560,21464.666667,32930.907947
43153,!13853,лютий,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,...,0.0,2.0,10.0,400.000000,0.0,565000.0,0.00820,5181.00600,21538.000000,32947.450300
43154,!13853,березень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,8.0,266.666667,65013.333333,...,55555.0,2.0,0.0,66.666667,0.0,3627500.0,0.00805,30186.57425,21464.666667,190080.145379
43155,!13853,квітень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,...,0.0,2.0,6.0,266.666667,0.0,565000.0,0.00790,4991.45700,21508.666667,32929.839517
43156,!13853,травень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,...,0.0,2.0,0.0,66.666667,0.0,565000.0,0.00780,4928.27400,21464.666667,32914.480367
43157,!13853,червень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,...,0.0,2.0,0.0,66.666667,0.0,627500.0,0.00770,5346.34100,21464.666667,36060.383717
43158,!13853,липень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,14.0,466.666667,64813.333333,...,0.0,2.0,3.0,166.666667,0.0,560000.0,0.00770,4826.59100,21486.666667,32665.496217
43159,!13853,серпень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,...,0.0,2.0,0.0,66.666667,0.0,560000.0,0.00780,4889.27400,21464.666667,32662.530367
43160,!13853,вересень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,...,0.0,2.0,0.0,66.666667,0.0,622500.0,0.00790,5445.70700,21464.666667,35815.352017
43161,!13853,жовтень,Сурмило Вікторія Володимирівна,0.46.,Управління соціальних та екологічних ризиків,33.333333,65280.0,0.0,0.000000,65280.000000,...,0.0,2.0,0.0,66.666667,0.0,560000.0,0.00790,4951.95700,21464.666667,32665.664517


### Для зберігання остаточного розрахунку необхідно здійснити заміну імен вимірів

In [77]:
df_staff = df_staff.rename(columns={'tabNum': 'Таб №',
                                        'avg_salary': 'Середня зп звітнього року',
                                        'period': 'Місяць бюджетного року',
                                        'Name': 'ПІБ',
                                        'CFOcode': 'ЦФУ код',
                                        'CFOname': 'ЦФУ назва',
                                        'bug_salary': '5.4.1.1.0.',
                                        'vacantion': 'Планові відпустки, дн. (в)',
                                        'bug_vacantion': '5.4.1.1.8.',
                                        'bug_gen_salary': '5.4.1.1.1.',
                                        'bug_add_salary': '5.4.1.1.2.',
                                        'add_salary_1': '5.4.1.1.3.3.',
                                        'add_salary_2': '5.4.1.1.3.5.',
                                        'bug_vac_reserv': '5.4.1.1.5.',
                                        'bug_work_argmnt': '5.4.1.1.4.',
                                        'bug_med_help': '5.4.1.3.',
                                        'bug_index': 'індекс (в)',
                                        'bug_indexing': '5.4.1.1.7.',
                                        'bug_gov_insurance': '5.4.1.2.1.',
                                        'bug_labour_union': '5.4.1.5.8.',
                                        })

### Попередній вигляд остаточного вигяду розрахунку перед зберіганням

In [78]:
print(df_staff.head(2))

  Таб № Місяць бюджетного року                          ПІБ   ЦФУ код  \
0    !8                 січень  Багатько Сергій Миколайович  1.19.15.   
1    !8                  лютий  Багатько Сергій Миколайович  1.19.15.   

                 ЦФУ назва  Середня зп звітнього року  5.4.1.1.0.  \
0  Операційний департамент                  33.333333         0.0   
1  Операційний департамент                  33.333333         0.0   

   Планові відпустки, дн. (в)  5.4.1.1.8.  5.4.1.1.1.  ...  5.4.1.1.3.5.  \
0                         0.0         0.0         0.0  ...           0.0   
1                         0.0         0.0         0.0  ...           0.0   

   bug_vac_reserv_1  bug_work_argmnt_soc  5.4.1.1.5.  5.4.1.1.4.  5.4.1.3.  \
0               0.0                  0.0         0.0         0.0  565000.0   
1               0.0                  0.0         0.0         0.0  565000.0   

   індекс (в)  5.4.1.1.7.  5.4.1.2.1.  5.4.1.5.8.  
0     0.00832    5046.912     21450.0  31654.8456  
1   

In [79]:
df_staff = df_staff.drop(['Середня зп звітнього року', 'Планові відпустки, дн. (в)', 'індекс (в)'], axis=1)

In [80]:
df_staff.columns

Index(['Таб №', 'Місяць бюджетного року', 'ПІБ', 'ЦФУ код', 'ЦФУ назва',
       '5.4.1.1.0.', '5.4.1.1.8.', '5.4.1.1.1.', '5.4.1.1.2.', '5.4.1.1.3.3.',
       '5.4.1.1.3.5.', 'bug_vac_reserv_1', 'bug_work_argmnt_soc', '5.4.1.1.5.',
       '5.4.1.1.4.', '5.4.1.3.', '5.4.1.1.7.', '5.4.1.2.1.', '5.4.1.5.8.'],
      dtype='object')

##**Зберігання результуючого датасету до папки "Out"**

In [81]:
path_out = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/Out/fop.xlsx'

In [82]:
df_staff.to_excel(path_out)

In [83]:
print("Calculation Done! 🎉 ")

Calculation Done! 🎉 


##**Форматування файлу для SA**

In [84]:
path_in ='/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/Out/fop.xlsx'

In [85]:
sheet_df_dictonary = pd.read_excel(path_in, engine='openpyxl', sheet_name=['Sheet1'], skiprows=0)

In [86]:
Sheet1 = sheet_df_dictonary['Sheet1'].drop(['Unnamed: 0'], axis=1)

In [87]:
print(Sheet1.columns)

Index(['Таб №', 'Місяць бюджетного року', 'ПІБ', 'ЦФУ код', 'ЦФУ назва',
       '5.4.1.1.0.', '5.4.1.1.8.', '5.4.1.1.1.', '5.4.1.1.2.', '5.4.1.1.3.3.',
       '5.4.1.1.3.5.', 'bug_vac_reserv_1', 'bug_work_argmnt_soc', '5.4.1.1.5.',
       '5.4.1.1.4.', '5.4.1.3.', '5.4.1.1.7.', '5.4.1.2.1.', '5.4.1.5.8.'],
      dtype='object')


In [88]:
def fResultForUploadSA(vArticleCode, vArticleName):
  # Виборка статті для агрегації
  df_new = Sheet1[['Місяць бюджетного року', 'ЦФУ код', 'ЦФУ назва', vArticleCode]].copy()
  # Початковий датафрейм
  df_tmp = df_new[df_new['Місяць бюджетного року'] == 'січень']
  result = df_tmp.groupby(['ЦФУ код', 'ЦФУ назва'])[vArticleCode].aggregate('sum').to_frame().reset_index()
  #
  result.insert(0, 'Назва статті', vArticleName)
  result.insert(0, 'Код статті', vArticleCode)
  # 
  result_total = result.rename(columns = {vArticleCode:'січень'})
  #
  months = ['лютий', 'березень', 'квітень', 'травень', 'червень', 'липень', 'серпень', 'вересень', 'жовтень', 'листопад', 'грудень']
  for i in months:
    df_new_tmp = df_new[df_new['Місяць бюджетного року'] == i]
    result = df_new_tmp.groupby(['ЦФУ код', 'ЦФУ назва'])[vArticleCode].aggregate('sum').to_frame().reset_index()
    result.insert(0, 'Назва статті', vArticleName)
    result.insert(0, 'Код статті', vArticleCode)
    df_tmp = result.rename(columns = {vArticleCode: i})
    result_total = pd.merge(result_total, df_tmp)

  return result_total

In [89]:
cluster_1 = fResultForUploadSA('5.4.1.1.0.','основна заробітна плата (оклади) - бюджет')
cluster_2 = fResultForUploadSA('5.4.1.1.8.','Виплачені відпустки (3-й клас) - бюджет')
cluster_3 = fResultForUploadSA('5.4.1.1.1.','основна заробітна плата')
cluster_4 = fResultForUploadSA('5.4.1.1.2.','додаткова заробітна плата')
cluster_5 = fResultForUploadSA('5.4.1.1.3.3.','премія за підсумками роботи Банку (за фінансовий рік)')
cluster_6 = fResultForUploadSA('5.4.1.1.3.5.','інші премії')
cluster_7 = fResultForUploadSA('5.4.1.1.5.','резерв відпусток')
cluster_8 = fResultForUploadSA('5.4.1.1.4.','трудові угоди')
cluster_9 = fResultForUploadSA('5.4.1.3.','мат. допомога і інші утримання')
cluster_10 = fResultForUploadSA('5.4.1.1.7.','індексація')
cluster_11 = fResultForUploadSA('5.4.1.2.1.','Обов\'язкове державне пенсійне та соціальне страхування')
cluster_12 = fResultForUploadSA('5.4.1.5.8.','профспілкові відрахування')

In [90]:
pdList = [cluster_1, cluster_2, cluster_3, cluster_4, cluster_5, cluster_6, cluster_7, cluster_8, cluster_9, cluster_10, cluster_11, cluster_12]
table = pd.concat(pdList)

## Доповнення стовбців для шаблону

In [91]:
table.insert(4, 'грудень 2022', '')
table.insert(4, 'Код проекта','')
table.insert(2, 'Код бізнеса','')
table.insert(2, 'Валюта', '[UAH]')
table.insert(2, 'Назва підрозділу', '')
table.insert(2, 'Код підрозділу', '')
table.insert(2, 'Назва дирекції','')
table.insert(2, 'Код дирекції','')
table.insert(0, 'Назва ініціатора','')
table.insert(0, 'Код ініціатора','1.42.')
table.insert(0, 'Назва адміністратора','')
table.insert(0, 'Код адміністратора', '1.43.')

In [92]:
# table.head()

In [93]:
path_out = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/Out/fop_SA.xlsx'

In [94]:
table.to_excel(path_out, index=False)

In [95]:
print("Formatting Done! 🎉 ")

Formatting Done! 🎉 
